In [2]:
!unzip  '/content/drive/MyDrive/imagenette2-320.zip'

Streaming output truncated to the last 5000 lines.
  inflating: imagenette2-320/train/n03445777/n03445777_8446.JPEG  
  inflating: imagenette2-320/train/n03445777/n03445777_8448.JPEG  
  inflating: imagenette2-320/train/n03445777/n03445777_8474.JPEG  
  inflating: imagenette2-320/train/n03445777/n03445777_8477.JPEG  
  inflating: imagenette2-320/train/n03445777/n03445777_8484.JPEG  
  inflating: imagenette2-320/train/n03445777/n03445777_8544.JPEG  
  inflating: imagenette2-320/train/n03445777/n03445777_8546.JPEG  
  inflating: imagenette2-320/train/n03445777/n03445777_8599.JPEG  
  inflating: imagenette2-320/train/n03445777/n03445777_8616.JPEG  
  inflating: imagenette2-320/train/n03445777/n03445777_8624.JPEG  
  inflating: imagenette2-320/train/n03445777/n03445777_8628.JPEG  
  inflating: imagenette2-320/train/n03445777/n03445777_866.JPEG  
  inflating: imagenette2-320/train/n03445777/n03445777_8663.JPEG  
  inflating: imagenette2-320/train/n03445777/n03445777_8677.JPEG  
  inflating:

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf


def read_dataset(path, size):
    # Define image data generator with augmentation options if needed
    image_generator = ImageDataGenerator()

    # Set up the image generator to flow from the directory
    batch_size = 32
    image_size = (size, size)  # adjust the size as needed
    class_mode = 'categorical'  # or 'binary' if it's a binary classification

    # Flow images in batches from the directory
    data_generator = image_generator.flow_from_directory(
        path,
        target_size=image_size,
        batch_size=batch_size,
        class_mode=class_mode
    )
    return data_generator


def get_train_dataset(size):
    path = '/content/imagenette2-320/train'
    return read_dataset(path, size)


def get_validation_dataset(size):
    path = '/content/imagenette2-320/val'
    return read_dataset(path, size)


In [4]:
# import dataset_loader
import tensorflow as tf
from sklearn.metrics import top_k_accuracy_score
from tensorflow import keras
from keras.models import Sequential
from tensorflow.keras.layers import Input, Conv2D, Dense, Flatten, Dropout

In [6]:
epochs = 1

train_data = get_train_dataset(256)
test_data = get_validation_dataset(256)

Found 9469 images belonging to 10 classes.
Found 3925 images belonging to 10 classes.


In [11]:
# Evaluation on VGG16 model with kernel size 5*5 in last layer instead of 7*7
pretrained_model = tf.keras.applications.vgg16.VGG16(
    include_top=False,
    weights='imagenet',
    classes=10,
    classifier_activation='softmax'
  )

for layer in pretrained_model.layers:
  layer.trainable = False

x = pretrained_model.output
x = Conv2D(4096, (5, 5), activation='relu', padding='valid')(x)
x = Conv2D(4096, (1,1), activation='relu', padding='valid')(x)
x = Conv2D(10, (1, 1), activation='softmax')(x)
x = tf.keras.layers.GlobalMaxPooling2D()(x)
x = Flatten()(x)

# Create a new model
model = tf.keras.Model(inputs=pretrained_model.input, outputs=x)
model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(learning_rate=1e-5), metrics=[keras.metrics.TopKCategoricalAccuracy(k=1, name='top 1%'), keras.metrics.TopKCategoricalAccuracy(k=5, name='top 5%')])

score = model.fit(train_data,validation_data=test_data, epochs=epochs)

296/296 [==============================] - 107s 352ms/step - loss: 0.5214 - top 1%: 0.8886 - top 5%: 0.9856 - val_loss: 0.3153 - val_top 1%: 0.9409 - val_top 5%: 0.9975


In [13]:
# Evaluation on VGG16 model with kernel size 3*3 in last layer instead of 7*7
pretrained_model = tf.keras.applications.vgg16.VGG16(
    include_top=False,
    weights='imagenet',
    classes=10,
    classifier_activation='softmax'
  )

for layer in pretrained_model.layers:
  layer.trainable = False

x = pretrained_model.output
x = Conv2D(4096, (3,3), activation='relu', padding='valid')(x)
x = Conv2D(4096, (1,1), activation='relu', padding='valid')(x)
x = Conv2D(10, (1, 1), activation='softmax')(x)
x = tf.keras.layers.GlobalMaxPooling2D()(x)
x = Flatten()(x)

# Create a new model
model = tf.keras.Model(inputs=pretrained_model.input, outputs=x)
model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(learning_rate=1e-5), metrics=[keras.metrics.TopKCategoricalAccuracy(k=1, name='top 1%'), keras.metrics.TopKCategoricalAccuracy(k=5, name='top 5%')])

score = model.fit(train_data,validation_data=test_data, epochs=5)

Epoch 1/5
296/296 [==============================] - 92s 306ms/step - loss: 1.0850 - top 1%: 0.5236 - top 5%: 0.9705 - val_loss: 0.9054 - val_top 1%: 0.6683 - val_top 5%: 0.9934
Epoch 2/5
296/296 [==============================] - 91s 308ms/step - loss: 0.7481 - top 1%: 0.6623 - top 5%: 0.9997 - val_loss: 0.8992 - val_top 1%: 0.6476 - val_top 5%: 0.9954
Epoch 3/5
296/296 [==============================] - 111s 374ms/step - loss: 0.6723 - top 1%: 0.6743 - top 5%: 1.0000 - val_loss: 0.9396 - val_top 1%: 0.6535 - val_top 5%: 0.9929
Epoch 4/5
296/296 [==============================] - 93s 312ms/step - loss: 0.6467 - top 1%: 0.6791 - top 5%: 1.0000 - val_loss: 1.0953 - val_top 1%: 0.6148 - val_top 5%: 0.9926
Epoch 5/5
296/296 [==============================] - 91s 309ms/step - loss: 0.6341 - top 1%: 0.6804 - top 5%: 1.0000 - val_loss: 1.0281 - val_top 1%: 0.6382 - val_top 5%: 0.9931


In [14]:
# Evaluation on VGG19 model with kernel size 5*5 in last layer instead of 7*7
pretrained_model = tf.keras.applications.vgg19.VGG19(
    include_top=False,
    weights='imagenet',
    classes=10,
    classifier_activation='softmax'
  )

for layer in pretrained_model.layers:
  layer.trainable = False

x = pretrained_model.output
x = Conv2D(4096, (5, 5), activation='relu', padding='valid')(x)
x = Conv2D(4096, (1,1), activation='relu', padding='valid')(x)
x = Conv2D(10, (1, 1), activation='softmax')(x)
x = tf.keras.layers.GlobalMaxPooling2D()(x)
x = Flatten()(x)

# Create a new model
model = tf.keras.Model(inputs=pretrained_model.input, outputs=x)
model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(learning_rate=1e-5), metrics=[keras.metrics.TopKCategoricalAccuracy(k=1, name='top 1%'), keras.metrics.TopKCategoricalAccuracy(k=5, name='top 5%')])

score = model.fit(train_data,validation_data=test_data, epochs=epochs)

296/296 [==============================] - 117s 391ms/step - loss: 0.5079 - top 1%: 0.8921 - top 5%: 0.9850 - val_loss: 0.3470 - val_top 1%: 0.9195 - val_top 5%: 0.9964


In [15]:
# Evaluation on VGG19 model with kernel size 3*3 in last layer instead of 7*7
pretrained_model = tf.keras.applications.vgg19.VGG19(
    include_top=False,
    weights='imagenet',
    classes=10,
    classifier_activation='softmax'
  )

for layer in pretrained_model.layers:
  layer.trainable = False

x = pretrained_model.output
x = Conv2D(4096, (3,3), activation='relu', padding='valid')(x)
x = Conv2D(4096, (1,1), activation='relu', padding='valid')(x)
x = Conv2D(10, (1, 1), activation='softmax')(x)
x = tf.keras.layers.GlobalMaxPooling2D()(x)
x = Flatten()(x)

# Create a new model
model = tf.keras.Model(inputs=pretrained_model.input, outputs=x)
model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(learning_rate=1e-5), metrics=[keras.metrics.TopKCategoricalAccuracy(k=1, name='top 1%'), keras.metrics.TopKCategoricalAccuracy(k=5, name='top 5%')])

score = model.fit(train_data,validation_data=test_data, epochs=5)

Epoch 1/5
296/296 [==============================] - 120s 401ms/step - loss: 1.0554 - top 1%: 0.5310 - top 5%: 0.9744 - val_loss: 0.9142 - val_top 1%: 0.5725 - val_top 5%: 0.9903
Epoch 2/5
296/296 [==============================] - 105s 355ms/step - loss: 0.7350 - top 1%: 0.6209 - top 5%: 0.9990 - val_loss: 0.8753 - val_top 1%: 0.6713 - val_top 5%: 0.9918
Epoch 3/5
296/296 [==============================] - 122s 414ms/step - loss: 0.6753 - top 1%: 0.6414 - top 5%: 0.9999 - val_loss: 0.9719 - val_top 1%: 0.5875 - val_top 5%: 0.9916
Epoch 4/5
296/296 [==============================] - 121s 409ms/step - loss: 0.6540 - top 1%: 0.6497 - top 5%: 1.0000 - val_loss: 1.1111 - val_top 1%: 0.5832 - val_top 5%: 0.9906
Epoch 5/5
296/296 [==============================] - 106s 358ms/step - loss: 0.6405 - top 1%: 0.6628 - top 5%: 1.0000 - val_loss: 1.1014 - val_top 1%: 0.5809 - val_top 5%: 0.9913
